In [1]:
# app.py

class Player:
    def __init__(self, username):
        self.username = username

class Piece:
    def __init__(self, color, type):
        self.color = color  # 'white' or 'black'
        self.type = type  # 'Pawn', 'Knight', 'Bishop', 'Rook', 'Queen', 'King'

class Board:
    def __init__(self):
        self.grid = self.initialize_board()
    
    def initialize_board(self):
        # Initialize the chessboard with pieces at the starting positions
        grid = [[None for _ in range(8)] for _ in range(8)]
        # Place Pawns
        for i in range(8):
            grid[1][i] = Piece('white', 'Pawn')
            grid[6][i] = Piece('black', 'Pawn')
        # Place Rooks
        grid[0][0] = grid[0][7] = Piece('white', 'Rook')
        grid[7][0] = grid[7][7] = Piece('black', 'Rook')
        # Place Knights
        grid[0][1] = grid[0][6] = Piece('white', 'Knight')
        grid[7][1] = grid[7][6] = Piece('black', 'Knight')
        # Place Bishops
        grid[0][2] = grid[0][5] = Piece('white', 'Bishop')
        grid[7][2] = grid[7][5] = Piece('black', 'Bishop')
        # Place Queens
        grid[0][3] = Piece('white', 'Queen')
        grid[7][3] = Piece('black', 'Queen')
        # Place Kings
        grid[0][4] = Piece('white', 'King')
        grid[7][4] = Piece('black', 'King')
        return grid

    def display(self):
        for row in self.grid:
            print(' '.join([piece.type[0] if piece else '.' for piece in row]))

class Game:
    def __init__(self, player1, player2):
        self.player1 = player1
        self.player2 = player2
        self.board = Board()
        self.turn = player1  # Player1 starts the game
        self.history = []

    def make_move(self, src, dst):
        piece = self.board.grid[src[0]][src[1]]
        if not piece:
            return "Invalid move: No piece at source position"
        if self.turn != self.get_player_by_color(piece.color):
            return "Invalid move: Not your turn"
        # Validate move based on piece type (simplified for brevity)
        # Move piece
        self.board.grid[dst[0]][dst[1]] = piece
        self.board.grid[src[0]][src[1]] = None
        self.history.append((src, dst))
        self.turn = self.player1 if self.turn == self.player2 else self.player2
        return "Move successful"

    def get_player_by_color(self, color):
        return self.player1 if self.player1.color == color else self.player2

from flask import Flask, request, jsonify

app = Flask(__name__)

# In-memory storage for users and games (for simplicity)
users = {}
games = {}

@app.route('/register', methods=['POST'])
def register():
    data = request.get_json()
    username = data.get('username')
    if username in users:
        return jsonify({"error": "User already exists"}), 400
    users[username] = Player(username)
    return jsonify({"message": "User registered successfully"}), 200

@app.route('/create_game', methods=['POST'])
def create_game():
    data = request.get_json()
    player1 = users.get(data.get('player1'))
    player2 = users.get(data.get('player2'))
    if not player1 or not player2:
        return jsonify({"error": "Invalid players"}), 400
    game_id = len(games) + 1
    games[game_id] = Game(player1, player2)
    return jsonify({"game_id": game_id}), 200

@app.route('/make_move', methods=['POST'])
def make_move():
    data = request.get_json()
    game_id = data.get('game_id')
    src = data.get('src')
    dst = data.get('dst')
    game = games.get(game_id)
    if not game:
        return jsonify({"error": "Invalid game ID"}), 400
    result = game.make_move(src, dst)
    return jsonify({"message": result}), 200

@app.route('/game_state', methods=['GET'])
def game_state():
    game_id = request.args.get('game_id')
    game = games.get(game_id)
    if not game:
        return jsonify({"error": "Invalid game ID"}), 400
    return jsonify({
        "board": [[piece.type if piece else None for piece in row] for row in game.board.grid],
        "turn": game.turn.username,
        "history": game.history
    }), 200

if __name__ == '__main__':
    app.run(debug=True)


 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
 * Restarting with watchdog (windowsapi)


SystemExit: 1

C:\Users\binny\anaconda3\Lib\site-packages\IPython\core\interactiveshell.py:3534: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
